In [2]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import pandas as pd, pickle, sys, itertools, string, sys, re, datetime, time, shutil, copy

# special matplotlib argument for improved plots
from matplotlib import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [3]:
#CARGAMOS LOS ARCHIVOS ORIGINALES
mtyid = pd.read_csv("MonterreyIDFAN.csv")
balid = pd.read_csv("BalconesIDFAN.csv")
mtyfd = pd.read_csv("MonterreyFUELDOSER.csv")
balfd = pd.read_csv("BalconesFUELDOSER.csv")

parosidbal = pd.read_csv("paros_IDFan_Balcones.csv")
paros=pd.read_csv("paros3.csv", encoding = "ISO-8859-1")


In [4]:
paros.columns.tolist()

['TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2']

In [5]:
#QUITAMOS LOS PAROS DE LA PLANTA DE PANAMA
paros=paros[paros["REG_DCPLANTA"]!="Panama"]
paros.to_csv("parosequipo2.csv", index=False)

In [6]:
#JUNTAMOS LA FECHA Y LA HORA REDONDEADA EN UNA COLUMNA APARTE LLAMADA "fechainicialred"
fechahora = paros["FECHA_INICIO"].astype(str) + "-" + paros["Hora_Ini_Red"].astype(str)
paros["fechainicialred"] = pd.to_datetime(fechahora)

In [7]:
paros.columns.tolist()

['TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred']

## MONTERREY IDFAN

In [8]:
mtyid = pd.read_csv("MonterreyIDFAN.csv")

In [9]:
#FILTRAMOS SOLO LOS PAROS QUE TENGAN QUE VER CON EL IDFAN EN MONTERREY
parosmtyid = paros[paros["FECHA_FIN.1"].isin(["2016.11.18 16:59:27", "2017.11.20 08:10:56","2017.11.29 09:01:41", "2018.08.06 18:16:26" ])]

In [10]:
parosmtyid.head()

,TPO_PKTIEMPO,REG_PKPLANTA,REG_DCPLANTA,EQU_PKEQUIPO,EQU_DCTIPOEQUIPOCWEN,EQU_DCEQUIPOCORTO,CPA_PKCAUSA_PARO,PAR_DCESTIPOPAROCW,PAR_DCCATEGORIAPAROCWSP,PAR_DCCOMENTARIO,FECHA_INICIO,Hora inicio,Hora_Ini_Red,FECHA_FIN,FECHA_FIN.1,FECHA_INICIO.1,FECHA_FIN.2,fechainicialred
1,18/11/2016,EA20,Monterrey,EA20-KL10,Kiln,KL10,EA20-32,NaN,NaN,SOBRECARGA VTI PRINCIPAL AL REALIZAR CAMBIO DE...,2016.11.18,14:20:42,14:30:00,2016.11.18 14:54:57,2016.11.18 16:59:27,NaN,NaN,2016-11-18 14:30:00
62,20/11/2017,EA20,Monterrey,EA20-KL10,Kiln,KL10,EA20-32,NaN,NaN,DESPRENDERSE MORRO DEL DUCTO DE SALIDA DE GAS ...,2017.11.20,03:36:56,03:30:00,2017.11.20 08:10:56,2017.11.20 08:10:56,NaN,NaN,2017-11-20 03:30:00
63,29/11/2017,EA20,Monterrey,EA20-KL10,Kiln,KL10,EA20-22,NaN,NaN,FALLAR RELEVADOR 737 QUE ALIMENTA A LOS VTI´S,2017.11.29,08:34:11,08:30:00,2017.11.29 09:01:41,2017.11.29 09:01:41,NaN,NaN,2017-11-29 08:30:00
106,06/08/2018,EA20,Monterrey,EA20-KL10,Kiln,KL10,EA20-22,NaN,NaN,PARAR VTI POR OPERAR PROTECCIÓN DE ALTA TEMPER...,2018.08.06,17:36:11,17:30:00,2018.08.06 18:16:26,2018.08.06 18:16:26,NaN,NaN,2018-08-06 17:30:00


In [11]:
#SE CREA UNA PIVOT TABLE PARA LOS TAGS DEL ID FAN DE MONTERREY, QUEDANDONOS COMO VARIABLES PREDICTORAS
mtyidfan=pd.crosstab(mtyid["date"], mtyid["description"], values=mtyid["value"], aggfunc="mean")
mtyidfan= mtyidfan.reset_index()

#SEPARAMOS LA FECHA COMPLETA EN COLUMNAS, PARA QUE EL MODELO PUEDA DESCRIBIRSE MEJOR
mtyidfan['ano'] = pd.DatetimeIndex(mtyidfan['date']).year
mtyidfan['mes'] = pd.DatetimeIndex(mtyidfan['date']).month
mtyidfan['dia'] = pd.DatetimeIndex(mtyidfan['date']).day
mtyidfan['hora']= pd.DatetimeIndex(mtyidfan['date']).hour
mtyidfan['minuto']= pd.DatetimeIndex(mtyidfan['date']).minute

#CONVERTIMOS LA COLUMNA DATE A datetime64[ns]
mtyidfan["date"]=mtyidfan["date"].astype("datetime64[ns]")

In [12]:
#SE VERIFICAN LOS TIPOS DE DATOS ANTES DEL MERGE
mtyidfan.dtypes

description
date                                       datetime64[ns]
 CHUM L. LIBRE VTI OTE HORNO 10                   float64
POTENCIA VTI OTE HORNO 10                         float64
POTENCIA VTI PTE HORNO 10                         float64
PRESION ENTRADA DRACCO HORNO 10                   float64
PRESION ENTRADA VTIS HORNO 10                     float64
TEMP CHUM L CARGA VTI OTE HORNO 10                float64
TEMP CHUM L CARGA VTI PTE HORNO 10                float64
TEMP CHUM L LIBRE VTI PTE HORNO 10                float64
TEMP SALIDA PRECALENTADOR HORNO 10                float64
Temp Chumacera Motor Lado Carga VTI OTE           float64
Temp Chumacera Motor Lado Libre VTI OTE           float64
VELOCIDAD VTI ORIENTE HORNO10                     float64
VELOCIDAD VTI PONIENTE HORNO10                    float64
VIBRACION EN VTI OTE HORNO 10                     float64
VIBRACION EN VTI PTE HORNO 10                     float64
ano                                                 int64
me

In [13]:
#SE HACE UN MERGE DE LOS DATOS CON TAGS JUNTO CON LOS PAROS DE SOLO ID FAN DE MONTERREY

mtyidparos = mtyidfan.merge(parosmtyid, how= "left", left_on= "date", right_on= "fechainicialred")

In [14]:
# CREAMOS UNA NUEVA COLUMNA QUE NOS INDICARÁ SI HUBO PARO EN ESA FECHA O NO CON 1 Y 0, RESPECTIVAMENTE

mtyidparos.loc[mtyidparos['FECHA_INICIO'].isna(), 'paro'] = 0
mtyidparos.loc[mtyidparos['FECHA_INICIO'].notnull(), 'paro'] = 1

#CONVERTIMOS LA COLUMNA PARO A INT64
mtyidparos["paro"]=mtyidparos["paro"].astype("int64")

mtyidparos.head()

,date,CHUM L. LIBRE VTI OTE HORNO 10,POTENCIA VTI OTE HORNO 10,POTENCIA VTI PTE HORNO 10,PRESION ENTRADA DRACCO HORNO 10,PRESION ENTRADA VTIS HORNO 10,TEMP CHUM L CARGA VTI OTE HORNO 10,TEMP CHUM L CARGA VTI PTE HORNO 10,TEMP CHUM L LIBRE VTI PTE HORNO 10,TEMP SALIDA PRECALENTADOR HORNO 10,...,PAR_DCCOMENTARIO,FECHA_INICIO,Hora inicio,Hora_Ini_Red,FECHA_FIN,FECHA_FIN.1,FECHA_INICIO.1,FECHA_FIN.2,fechainicialred,paro
0,2016-10-01 00:00:00,56.65437,940.4066,889.2732,42.12796,617.6624,56.05050,63.53107,60.06686,331.8810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
1,2016-10-01 00:30:00,56.61032,919.8439,899.3297,40.69216,644.2539,56.03790,63.51220,59.98492,332.1586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
2,2016-10-01 01:00:00,56.56627,935.7388,938.7916,38.96289,621.0678,56.02531,63.49777,59.90299,332.4363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
3,2016-10-01 01:30:00,56.52222,924.4288,943.7044,46.99400,622.4374,56.01271,63.49148,59.82106,334.5285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
4,2016-10-01 02:00:00,56.45634,927.0964,922.0532,48.81271,652.3612,56.00011,63.48518,59.73912,333.6945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0


In [15]:
mtyidparos.columns.tolist()

['date',
 ' CHUM L. LIBRE VTI OTE HORNO 10',
 'POTENCIA VTI OTE HORNO 10',
 'POTENCIA VTI PTE HORNO 10',
 'PRESION ENTRADA DRACCO HORNO 10',
 'PRESION ENTRADA VTIS HORNO 10',
 'TEMP CHUM L CARGA VTI OTE HORNO 10',
 'TEMP CHUM L CARGA VTI PTE HORNO 10',
 'TEMP CHUM L LIBRE VTI PTE HORNO 10',
 'TEMP SALIDA PRECALENTADOR HORNO 10',
 'Temp Chumacera Motor Lado Carga VTI OTE',
 'Temp Chumacera Motor Lado Libre VTI OTE',
 'VELOCIDAD VTI ORIENTE HORNO10',
 'VELOCIDAD VTI PONIENTE HORNO10',
 'VIBRACION EN VTI OTE HORNO 10',
 'VIBRACION EN VTI PTE HORNO 10',
 'ano',
 'mes',
 'dia',
 'hora',
 'minuto',
 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred',
 'paro']

In [16]:
#SE QUITAN TODAS LAS COLUMNAS QUE NO NOS SERVIRÁN PARA EL MODELO (LAS DEL ARCHIVO DE PAROS)
mtyidparos = mtyidparos.drop([ 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred'], axis=1)


In [17]:
#CHECAMOS VALORES NULOS
mtyidparos.isnull().sum()

date                                        0
 CHUM L. LIBRE VTI OTE HORNO 10             0
POTENCIA VTI OTE HORNO 10                   2
POTENCIA VTI PTE HORNO 10                   2
PRESION ENTRADA DRACCO HORNO 10            82
PRESION ENTRADA VTIS HORNO 10              82
TEMP CHUM L CARGA VTI OTE HORNO 10          0
TEMP CHUM L CARGA VTI PTE HORNO 10          0
TEMP CHUM L LIBRE VTI PTE HORNO 10          0
TEMP SALIDA PRECALENTADOR HORNO 10          0
Temp Chumacera Motor Lado Carga VTI OTE    50
Temp Chumacera Motor Lado Libre VTI OTE    50
VELOCIDAD VTI ORIENTE HORNO10              69
VELOCIDAD VTI PONIENTE HORNO10             69
VIBRACION EN VTI OTE HORNO 10              50
VIBRACION EN VTI PTE HORNO 10              50
ano                                         0
mes                                         0
dia                                         0
hora                                        0
minuto                                      0
paro                              

In [18]:
#ELIMINAMOS LAS FILAS CON VALORES NULOS PUES SON MUY POCAS
mtyidparos=mtyidparos.dropna()

In [19]:
#VOLVEMOS A CHECAR
mtyidparos.isnull().sum()

date                                       0
 CHUM L. LIBRE VTI OTE HORNO 10            0
POTENCIA VTI OTE HORNO 10                  0
POTENCIA VTI PTE HORNO 10                  0
PRESION ENTRADA DRACCO HORNO 10            0
PRESION ENTRADA VTIS HORNO 10              0
TEMP CHUM L CARGA VTI OTE HORNO 10         0
TEMP CHUM L CARGA VTI PTE HORNO 10         0
TEMP CHUM L LIBRE VTI PTE HORNO 10         0
TEMP SALIDA PRECALENTADOR HORNO 10         0
Temp Chumacera Motor Lado Carga VTI OTE    0
Temp Chumacera Motor Lado Libre VTI OTE    0
VELOCIDAD VTI ORIENTE HORNO10              0
VELOCIDAD VTI PONIENTE HORNO10             0
VIBRACION EN VTI OTE HORNO 10              0
VIBRACION EN VTI PTE HORNO 10              0
ano                                        0
mes                                        0
dia                                        0
hora                                       0
minuto                                     0
paro                                       0
dtype: int

In [20]:
#GUARDAMOS NUESTRO ARCHIVO PREPARADO PARA APLICAR UN MODELO DE MACHINE LEARNING
mtyidparos.to_csv("mtyIDFANconparos.csv", index=False)

# MONTERREY FUELDOSER

In [135]:
mtyfd = pd.read_csv("MonterreyFUELDOSER.csv")

In [ ]:
#FILTRAMOS SOLO LOS PAROS QUE TENGAN QUE VER CON EL FUELDOSER EN MONTERREY
parosmtyfd = paros[paros["FECHA_FIN.1"].isin(["2016.11.18 16:59:27", "2017.11.20 08:10:56","2017.11.29 09:01:41", "2018.08.06 18:16:26" ])]

In [ ]:
parosmtyfd.head()

In [ ]:
#SE CREA UNA PIVOT TABLE PARA LOS TAGS DEL ID FAN DE MONTERREY, QUEDANDONOS COMO VARIABLES PREDICTORAS
mtyfueldoser=pd.crosstab(mtyfd["date"], mtyfd["description"], values=mtyfd["value"], aggfunc="mean")
mtyfueldoser= mtyfueldoser.reset_index()

#SEPARAMOS LA FECHA COMPLETA EN COLUMNAS, PARA QUE EL MODELO PUEDA DESCRIBIRSE MEJOR
mtyfueldoser['ano'] = pd.DatetimeIndex(mtyfueldoser['date']).year
mtyfueldoser['mes'] = pd.DatetimeIndex(mtyfueldoser['date']).month
mtyfueldoser['dia'] = pd.DatetimeIndex(mtyfueldoser['date']).day
mtyfueldoser['hora']= pd.DatetimeIndex(mtyfueldoser['date']).hour
mtyfueldoser['minuto']= pd.DatetimeIndex(mtyfueldoser['date']).minute

#CONVERTIMOS LA COLUMNA DATE A datetime64[ns]
mtyfueldoser["date"]=mtyfueldoser["date"].astype("datetime64[ns]")

In [ ]:
#SE VERIFICAN LOS TIPOS DE DATOS ANTES DEL MERGE
mtyfueldoser.dtypes

In [ ]:
#SE HACE UN MERGE DE LOS DATOS CON TAGS JUNTO CON LOS PAROS DE SOLO FUELDOSER DE MONTERREY
mtyfdparos = mtyfueldoser.merge(parosmtyfd, how= "left", left_on= "date", right_on= "fechainicialred")

In [ ]:
# CREAMOS UNA NUEVA COLUMNA QUE NOS INDICARÁ SI HUBO PARO EN ESA FECHA O NO CON 1 Y 0, RESPECTIVAMENTE

mtyfdparos.loc[mtyfdparos['FECHA_INICIO'].isna(), 'paro'] = 0
mtyfdparos.loc[mtyfdparos['FECHA_INICIO'].notnull(), 'paro'] = 1

#CONVERTIMOS LA COLUMNA PARO A INT64
mtyfdparos["paro"]=mtyfdparos["paro"].astype("int64")

mtyfdparos.head()

In [ ]:
mtyfdparos.columns.tolist()

In [ ]:
#SE QUITAN TODAS LAS COLUMNAS QUE NO NOS SERVIRÁN PARA EL MODELO (LAS DEL ARCHIVO DE PAROS)
mtyfdparos = mtyfdparos.drop([ 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred'], axis=1)

In [ ]:
#CHECAMOS VALORES NULOS
mtyfdparos.isnull().sum()

In [ ]:
#ELIMINAMOS LAS FILAS CON VALORES NULOS PUES SON MUY POCAS
mtyfdparos=mtyfdparos.dropna()

In [ ]:
#VOLVEMOS A CHECAR
mtyfdparos.isnull().sum()

In [ ]:
#GUARDAMOS NUESTRO ARCHIVO PREPARADO PARA APLICAR UN MODELO DE MACHINE LEARNING
mtyfdparos.to_csv("mtyFUELDOSERconparos.csv", index=False)

# BALCONES IDFAN

In [136]:
balid = pd.read_csv("BalconesIDFAN.csv")

In [137]:
parosidbal["FECHA_FIN"].unique().tolist()

['2016.12.22 09:19:00',
 '2017.01.22 03:00:00',
 '2017.04.01 16:03:00',
 '2017.05.26 05:23:00',
 '2017.05.26 05:24:00',
 '2017.08.15 11:54:00',
 '2017.09.10 00:00:00',
 '2017.09.10 05:06:00',
 '2017.09.25 10:11:00',
 '2018.01.21 07:31:00',
 '2018.09.27 20:16:00',
 '2019.02.27 11:14:00',
 '2019.04.08 04:28:00',
 '2019.04.10 12:37:00',
 '2019.05.23 09:43:00',
 '2019.06.09 22:31:00',
 '2019.06.17 05:02:00']

In [138]:
#FILTRAMOS SOLO LOS PAROS QUE TENGAN QUE VER CON EL IDFAN EN BALCONES
parosbalid = paros[paros["FECHA_FIN"].isin(parosidbal["FECHA_FIN"].unique().tolist())]

In [139]:
parosbalid=parosbalid.iloc[:18]
parosbalid.head(20)

,TPO_PKTIEMPO,REG_PKPLANTA,REG_DCPLANTA,EQU_PKEQUIPO,EQU_DCTIPOEQUIPOCWEN,EQU_DCEQUIPOCORTO,CPA_PKCAUSA_PARO,PAR_DCESTIPOPAROCW,PAR_DCCATEGORIAPAROCWSP,PAR_DCCOMENTARIO,FECHA_INICIO,Hora inicio,Hora_Ini_Red,FECHA_FIN,FECHA_FIN.1,FECHA_INICIO.1,FECHA_FIN.2,fechainicialred
775,09/09/2017,140,Balcones,140-22,Raw Mill,RM2,140_22,Programa de Operación,Operativos,KILN DOWN,2017.09.09,21:34:00,21:30:00,2017.09.10 00:00:00,2017.09.10 00:00:00,NaN,NaN,2017-09-09 21:30:00
2130,22/12/2016,140,Balcones,140-32,Kiln,Kiln2,140_3,Falla Eléctrica,Mtto.Correctivo,COOLER BAGHOUSE FAN FAULT,2016.12.22,08:55:00,09:00:00,2016.12.22 09:19:00,2016.12.22 09:19:00,NaN,NaN,2016-12-22 09:00:00
2145,22/01/2017,140,Balcones,140-32,Kiln,Kiln2,140_3,Falla Eléctrica,Mtto.Correctivo,AIRSLIDE 3 BLOWER TURBO FAN MOTOR STOPPED BECA...,2017.01.22,00:06:00,00:00:00,2017.01.22 03:00:00,2017.01.22 03:00:00,NaN,NaN,2017-01-22 00:00:00
2169,01/04/2017,140,Balcones,140-32,Kiln,Kiln2,140_3,Falla Eléctrica,Mtto.Correctivo,"LOSS OF COMMS IN ER 569, COOLER FANS.",2017.04.01,14:39:00,14:30:00,2017.04.01 16:03:00,2017.04.01 16:03:00,NaN,NaN,2017-04-01 14:30:00
2198,26/05/2017,140,Balcones,140-32,Kiln,Kiln2,140_3,Falla Eléctrica,Mtto.Correctivo,MAIN BH FAN NOT READY,2017.05.26,03:23:00,03:30:00,2017.05.26 05:23:00,2017.05.26 05:23:00,NaN,NaN,2017-05-26 03:30:00
2200,26/05/2017,140,Balcones,140-32,Kiln,Kiln2,140_2021,NaN,NaN,TRANSITION; MAIN BAGHOUSE FAN TOOK THE KILN DO...,2017.05.26,05:23:00,05:30:00,2017.05.26 05:24:00,2017.05.26 05:24:00,NaN,NaN,2017-05-26 05:30:00
2212,15/08/2017,140,Balcones,140-32,Kiln,Kiln2,140_4,Falla Mecánica,Mtto.Correctivo,MAIN BH FAN ODE HIGH VIBRATION,2017.08.15,10:33:00,10:30:00,2017.08.15 11:54:00,2017.08.15 11:54:00,NaN,NaN,2017-08-15 10:30:00
2228,09/09/2017,140,Balcones,140-32,Kiln,Kiln2,140_3,Falla Eléctrica,Mtto.Correctivo,CLINKER COOLER FAN 5 DRIVE HIGH TEMP. DRIVE CA...,2017.09.09,21:34:00,21:30:00,2017.09.10 00:00:00,2017.09.10 00:00:00,NaN,NaN,2017-09-09 21:30:00
2229,10/09/2017,140,Balcones,140-32,Kiln,Kiln2,140_1003,NaN,NaN,CLINKER COOLER FAN 5 DRIVE HIGH TEMP. DRIVE CA...,2017.09.10,00:00:00,00:00:00,2017.09.10 05:06:00,2017.09.10 05:06:00,NaN,NaN,2017-09-10 00:00:00
2235,25/09/2017,140,Balcones,140-32,Kiln,Kiln2,140_5,Falla en la Operación,Operativos,ALL COOLER FANS STOPPED ON #4 FAN DOWN >5MIN I...,2017.09.25,09:53:00,10:00:00,2017.09.25 10:11:00,2017.09.25 10:11:00,NaN,NaN,2017-09-25 10:00:00


In [140]:
#SE CREA UNA PIVOT TABLE PARA LOS TAGS DEL ID FAN DE BALCONES, QUEDANDONOS COMO VARIABLES PREDICTORAS
balidfan=pd.crosstab(balid["date"], balid["description"], values=balid["value"], aggfunc="mean")
balidfan= balidfan.reset_index()

#SEPARAMOS LA FECHA COMPLETA EN COLUMNAS, PARA QUE EL MODELO PUEDA DESCRIBIRSE MEJOR
balidfan['ano'] = pd.DatetimeIndex(balidfan['date']).year
balidfan['mes'] = pd.DatetimeIndex(balidfan['date']).month
balidfan['dia'] = pd.DatetimeIndex(balidfan['date']).day
balidfan['hora']= pd.DatetimeIndex(balidfan['date']).hour
balidfan['minuto']= pd.DatetimeIndex(balidfan['date']).minute

#CONVERTIMOS LA COLUMNA DATE A datetime64[ns]
balidfan["date"]=balidfan["date"].astype("datetime64[ns]")

In [141]:
#SE VERIFICAN LOS TIPOS DE DATOS ANTES DEL MERGE
balidfan.dtypes

description
date                                      datetime64[ns]
KILN ID FAN MOTOR DE BRNG TEMP -KILN2            float64
KILN ID FAN MOTOR ODE BRNG TEMP -KILN2           float64
KILN ID FAN OUTLET PRESSURE -KILN2               float64
KILN ID FAN POWER -KILN2                         float64
KILN ID FAN SPEED                                float64
KILN ID FAN TEMP #1 FAN BEARING -KILN2           float64
KILN ID FAN TEMP #2 FAN BEARING -KILN2           float64
KILN ID FAN VIBRATION -KILN2                     float64
STAGE 1A DISCHARGE GAS PRESSURE -KILN2           float64
STAGE 1B DISCHARGE GAS PRESSURE -KILN2           float64
WATER SPRAY TOWER OUTLET TEMP 1 -KILN2           float64
WATER SPRAY TOWER OUTLET TEMP 2 -KILN2           float64
ano                                                int64
mes                                                int64
dia                                                int64
hora                                               int64
minuto             

In [142]:
#SE HACE UN MERGE DE LOS DATOS CON TAGS JUNTO CON LOS PAROS DE SOLO ID FAN DE BALCONES
balidparos = balidfan.merge(parosbalid, how= "left", left_on= "date", right_on= "fechainicialred")

In [143]:
# CREAMOS UNA NUEVA COLUMNA QUE NOS INDICARÁ SI HUBO PARO EN ESA FECHA O NO CON 1 Y 0, RESPECTIVAMENTE

balidparos.loc[balidparos['FECHA_INICIO'].isna(), 'paro'] = 0
balidparos.loc[balidparos['FECHA_INICIO'].notnull(), 'paro'] = 1

#CONVERTIMOS LA COLUMNA PARO A INT64
balidparos["paro"]=balidparos["paro"].astype("int64")

balidparos.head()

,date,KILN ID FAN MOTOR DE BRNG TEMP -KILN2,KILN ID FAN MOTOR ODE BRNG TEMP -KILN2,KILN ID FAN OUTLET PRESSURE -KILN2,KILN ID FAN POWER -KILN2,KILN ID FAN SPEED,KILN ID FAN TEMP #1 FAN BEARING -KILN2,KILN ID FAN TEMP #2 FAN BEARING -KILN2,KILN ID FAN VIBRATION -KILN2,STAGE 1A DISCHARGE GAS PRESSURE -KILN2,...,PAR_DCCOMENTARIO,FECHA_INICIO,Hora inicio,Hora_Ini_Red,FECHA_FIN,FECHA_FIN.1,FECHA_INICIO.1,FECHA_FIN.2,fechainicialred,paro
0,2016-10-01 00:00:00,122.3483,91.0,-1.991539,2283.381,773.7543,151.8130,150.9543,0.049514,-36.64047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
1,2016-10-01 00:30:00,122.7297,91.0,-1.988343,2281.579,773.8171,151.6228,150.7006,0.050583,-36.38768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
2,2016-10-01 01:00:00,123.1110,91.0,-2.143953,2309.477,773.5776,151.4327,150.4469,0.051653,-36.93771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
3,2016-10-01 01:30:00,123.4924,91.0,-2.036518,2297.987,773.6574,151.2425,150.1932,0.052723,-36.33629,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
4,2016-10-01 02:00:00,123.8737,91.0,-2.108116,2267.121,773.8047,151.0524,149.9395,0.053792,-36.78667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0


In [144]:
balidparos.columns.tolist()

['date',
 'KILN ID FAN MOTOR DE BRNG TEMP -KILN2',
 'KILN ID FAN MOTOR ODE BRNG TEMP -KILN2',
 'KILN ID FAN OUTLET PRESSURE -KILN2',
 'KILN ID FAN POWER -KILN2',
 'KILN ID FAN SPEED',
 'KILN ID FAN TEMP #1 FAN BEARING -KILN2',
 'KILN ID FAN TEMP #2 FAN BEARING -KILN2',
 'KILN ID FAN VIBRATION -KILN2',
 'STAGE 1A DISCHARGE GAS PRESSURE -KILN2',
 'STAGE 1B DISCHARGE GAS PRESSURE -KILN2',
 'WATER SPRAY TOWER OUTLET TEMP 1 -KILN2',
 'WATER SPRAY TOWER OUTLET TEMP 2 -KILN2',
 'ano',
 'mes',
 'dia',
 'hora',
 'minuto',
 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred',
 'paro']

In [145]:
#SE QUITAN TODAS LAS COLUMNAS QUE NO NOS SERVIRÁN PARA EL MODELO (LAS DEL ARCHIVO DE PAROS)
balidparos = balidparos.drop([ 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred'], axis=1)

In [148]:
#CHECAMOS VALORES NULOS
balidparos.isnull().sum()

date                                        0
KILN ID FAN MOTOR DE BRNG TEMP -KILN2       0
KILN ID FAN MOTOR ODE BRNG TEMP -KILN2      0
KILN ID FAN OUTLET PRESSURE -KILN2          0
KILN ID FAN POWER -KILN2                  390
KILN ID FAN SPEED                         344
KILN ID FAN TEMP #1 FAN BEARING -KILN2      0
KILN ID FAN TEMP #2 FAN BEARING -KILN2      0
KILN ID FAN VIBRATION -KILN2                0
STAGE 1A DISCHARGE GAS PRESSURE -KILN2      0
STAGE 1B DISCHARGE GAS PRESSURE -KILN2      0
WATER SPRAY TOWER OUTLET TEMP 1 -KILN2      0
WATER SPRAY TOWER OUTLET TEMP 2 -KILN2      0
ano                                         0
mes                                         0
dia                                         0
hora                                        0
minuto                                      0
paro                                        0
dtype: int64

In [149]:
#ELIMINAMOS LAS FILAS CON VALORES NULOS PUES SON MUY POCAS
balidparos=balidparos.dropna()

In [150]:
#VOLVEMOS A CHECAR
balidparos.isnull().sum()

date                                      0
KILN ID FAN MOTOR DE BRNG TEMP -KILN2     0
KILN ID FAN MOTOR ODE BRNG TEMP -KILN2    0
KILN ID FAN OUTLET PRESSURE -KILN2        0
KILN ID FAN POWER -KILN2                  0
KILN ID FAN SPEED                         0
KILN ID FAN TEMP #1 FAN BEARING -KILN2    0
KILN ID FAN TEMP #2 FAN BEARING -KILN2    0
KILN ID FAN VIBRATION -KILN2              0
STAGE 1A DISCHARGE GAS PRESSURE -KILN2    0
STAGE 1B DISCHARGE GAS PRESSURE -KILN2    0
WATER SPRAY TOWER OUTLET TEMP 1 -KILN2    0
WATER SPRAY TOWER OUTLET TEMP 2 -KILN2    0
ano                                       0
mes                                       0
dia                                       0
hora                                      0
minuto                                    0
paro                                      0
dtype: int64

In [151]:
#GUARDAMOS NUESTRO ARCHIVO PREPARADO PARA APLICAR UN MODELO DE MACHINE LEARNING
balidparos.to_csv("balIDFANconparos.csv", index=False)

# BALCONES FUELDOSER

In [152]:
balfd = pd.read_csv("BalconesFUELDOSER.csv")

In [153]:
parosfdbal = pd.read_csv("paros_FuelDoser_Balcones.csv")

In [154]:
parosfdbal["FECHA_FIN"].unique().tolist()

['2017.05.27 00:33:00',
 '2017.12.18 15:24:00',
 '2018.01.11 15:24:00',
 '2018.03.04 15:24:00',
 '2018.07.01 15:24:00',
 '2018.07.27 15:24:00',
 '2018.07.29 15:24:00',
 '2018.08.03 15:24:00',
 '2018.08.13 05:03:00',
 '2018.08.29 09:07:00',
 '2018.09.12 15:24:00',
 '2018.10.10 12:31:00',
 '2018.10.11 12:31:00',
 '2018.10.12 12:31:00',
 '2018.10.13 12:31:00',
 '2018.10.18 12:31:00',
 '2018.10.19 12:31:00',
 '2018.10.20 12:31:00',
 '2018.10.22 12:31:00',
 '2018.12.06 12:31:00',
 '2019.01.03 12:31:00',
 '2019.02.12 00:00:00',
 '2019.03.06 12:31:00',
 '2019.03.10 00:00:00',
 '2019.06.09 11:01:00',
 '2019.06.19 11:01:00',
 '2019.06.25 11:01:00',
 '2019.07.20 18:40:00']

In [155]:
#FILTRAMOS SOLO LOS PAROS QUE TENGAN QUE VER CON EL FUELDOSER EN BALCONES
parosbalfd = paros[paros["FECHA_FIN"].isin(parosfdbal["FECHA_FIN"].unique().tolist())]

In [157]:
parosbalfd.shape

(30, 18)

In [158]:
#SE CREA UNA PIVOT TABLE PARA LOS TAGS DEL FUELDOSER DE BALCONES, QUEDANDONOS COMO VARIABLES PREDICTORAS
balfueldoser=pd.crosstab(balfd["date"], balfd["description"], values=balfd["value"], aggfunc="mean")
balfueldoser= balfueldoser.reset_index()

#SEPARAMOS LA FECHA COMPLETA EN COLUMNAS, PARA QUE EL MODELO PUEDA DESCRIBIRSE MEJOR
balfueldoser['ano'] = pd.DatetimeIndex(balfueldoser['date']).year
balfueldoser['mes'] = pd.DatetimeIndex(balfueldoser['date']).month
balfueldoser['dia'] = pd.DatetimeIndex(balfueldoser['date']).day
balfueldoser['hora']= pd.DatetimeIndex(balfueldoser['date']).hour
balfueldoser['minuto']= pd.DatetimeIndex(balfueldoser['date']).minute

#CONVERTIMOS LA COLUMNA DATE A datetime64[ns]
balfueldoser["date"]=balfueldoser["date"].astype("datetime64[ns]")

In [159]:
#SE VERIFICAN LOS TIPOS DE DATOS ANTES DEL MERGE
balfueldoser.dtypes

description
date                                       datetime64[ns]
CALC. ROTOWFDR FEEDRATE -KILN2                    float64
CALC. ROTOWFDR LOADING MATERIAL -KILN2            float64
CALCINER BLOWER CURRENT -KILN2                    float64
CALCINER FEEDER BLOWER PRESSURE -KILN2            float64
FEED FLOWMETER ACTUAL FEEDRATE -KILN2             float64
FEED ROTOWFDR FEEDRATE -KILN2                     float64
K2 TIM FEEDRATE                                   float64
KILN BURNER WFDR SPEED -KILN2                     float64
KILN FEEDER BLOWER CURRENT -KILN2                 float64
KILN FEEDER BLOWER PRESSURE -KILN2                float64
KLN BURNER WFDR LOADING MATERIAL -KILN2           float64
KLN GAS BMS FLOW -KILN2                           float64
MAIN BURNR ROTARY SPEED - KILN/COOLER             float64
MAIN COKE FEED RATE -KILN2                        float64
PYROCLON DUCT TEMPERATURE -KILN2                  float64
SCHENCK WOOD TPH                                  float64
TE

In [160]:
#SE HACE UN MERGE DE LOS DATOS CON TAGS JUNTO CON LOS PAROS DE SOLO FUELDOSER DE BALCONES
balfdparos = balfueldoser.merge(parosbalfd, how= "left", left_on= "date", right_on= "fechainicialred")

In [161]:
# CREAMOS UNA NUEVA COLUMNA QUE NOS INDICARÁ SI HUBO PARO EN ESA FECHA O NO CON 1 Y 0, RESPECTIVAMENTE

balfdparos.loc[balfdparos['FECHA_INICIO'].isna(), 'paro'] = 0
balfdparos.loc[balfdparos['FECHA_INICIO'].notnull(), 'paro'] = 1

#CONVERTIMOS LA COLUMNA PARO A INT64
balfdparos["paro"]=balfdparos["paro"].astype("int64")

balfdparos.head()

,date,CALC. ROTOWFDR FEEDRATE -KILN2,CALC. ROTOWFDR LOADING MATERIAL -KILN2,CALCINER BLOWER CURRENT -KILN2,CALCINER FEEDER BLOWER PRESSURE -KILN2,FEED FLOWMETER ACTUAL FEEDRATE -KILN2,FEED ROTOWFDR FEEDRATE -KILN2,K2 TIM FEEDRATE,KILN BURNER WFDR SPEED -KILN2,KILN FEEDER BLOWER CURRENT -KILN2,...,PAR_DCCOMENTARIO,FECHA_INICIO,Hora inicio,Hora_Ini_Red,FECHA_FIN,FECHA_FIN.1,FECHA_INICIO.1,FECHA_FIN.2,fechainicialred,paro
0,2016-10-01 00:00:00,1.954624,117.5702,80.18141,1.801634,0.0,250.8236,2.043782,762.0124,85.81585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
1,2016-10-01 00:30:00,2.084685,116.1478,82.46400,1.823007,0.0,250.8170,2.057861,768.6467,88.45000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
2,2016-10-01 01:00:00,2.483863,121.6483,80.49825,1.844380,0.0,250.8104,2.061873,754.6323,87.67714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
3,2016-10-01 01:30:00,2.708896,118.1240,84.25157,1.865753,0.0,250.8038,2.019328,735.7861,88.15898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
4,2016-10-01 02:00:00,1.991432,118.2477,79.99693,1.887126,0.0,250.7972,1.882527,738.4516,85.74269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0


In [162]:
balfdparos.columns.tolist()

['date',
 'CALC. ROTOWFDR FEEDRATE -KILN2',
 'CALC. ROTOWFDR LOADING MATERIAL -KILN2',
 'CALCINER BLOWER CURRENT -KILN2',
 'CALCINER FEEDER BLOWER PRESSURE -KILN2',
 'FEED FLOWMETER ACTUAL FEEDRATE -KILN2',
 'FEED ROTOWFDR FEEDRATE -KILN2',
 'K2 TIM FEEDRATE',
 'KILN BURNER WFDR SPEED -KILN2',
 'KILN FEEDER BLOWER CURRENT -KILN2',
 'KILN FEEDER BLOWER PRESSURE -KILN2',
 'KLN BURNER WFDR LOADING MATERIAL -KILN2',
 'KLN GAS BMS FLOW -KILN2',
 'MAIN BURNR ROTARY SPEED - KILN/COOLER',
 'MAIN COKE FEED RATE -KILN2',
 'PYROCLON DUCT TEMPERATURE -KILN2',
 'SCHENCK WOOD TPH',
 'TERTIARY AIR TO CALCINER TEMP. -KILN2',
 'ano',
 'mes',
 'dia',
 'hora',
 'minuto',
 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialre

In [163]:
#SE QUITAN TODAS LAS COLUMNAS QUE NO NOS SERVIRÁN PARA EL MODELO (LAS DEL ARCHIVO DE PAROS)
balfdparos = balfdparos.drop([ 'TPO_PKTIEMPO',
 'REG_PKPLANTA',
 'REG_DCPLANTA',
 'EQU_PKEQUIPO',
 'EQU_DCTIPOEQUIPOCWEN',
 'EQU_DCEQUIPOCORTO',
 'CPA_PKCAUSA_PARO',
 'PAR_DCESTIPOPAROCW',
 'PAR_DCCATEGORIAPAROCWSP',
 'PAR_DCCOMENTARIO',
 'FECHA_INICIO',
 'Hora inicio',
 'Hora_Ini_Red',
 'FECHA_FIN',
 'FECHA_FIN.1',
 'FECHA_INICIO.1',
 'FECHA_FIN.2',
 'fechainicialred'], axis=1)

In [164]:
#CHECAMOS VALORES NULOS
balfdparos.isnull().sum()

date                                          0
CALC. ROTOWFDR FEEDRATE -KILN2                0
CALC. ROTOWFDR LOADING MATERIAL -KILN2        0
CALCINER BLOWER CURRENT -KILN2                0
CALCINER FEEDER BLOWER PRESSURE -KILN2        0
FEED FLOWMETER ACTUAL FEEDRATE -KILN2         0
FEED ROTOWFDR FEEDRATE -KILN2                 0
K2 TIM FEEDRATE                               0
KILN BURNER WFDR SPEED -KILN2                 0
KILN FEEDER BLOWER CURRENT -KILN2             1
KILN FEEDER BLOWER PRESSURE -KILN2            0
KLN BURNER WFDR LOADING MATERIAL -KILN2       0
KLN GAS BMS FLOW -KILN2                      21
MAIN BURNR ROTARY SPEED - KILN/COOLER      3386
MAIN COKE FEED RATE -KILN2                    0
PYROCLON DUCT TEMPERATURE -KILN2              0
SCHENCK WOOD TPH                              0
TERTIARY AIR TO CALCINER TEMP. -KILN2         0
ano                                           0
mes                                           0
dia                                     

In [166]:
#RELLENAMOS LAS COLUMNAS CON NUMERO DE VALORES NULOS CONSIDERABLES CON SU MISMO PROMEDIO
balfdparos["MAIN BURNR ROTARY SPEED - KILN/COOLER"] = balfdparos["MAIN BURNR ROTARY SPEED - KILN/COOLER"].fillna(balfdparos["MAIN BURNR ROTARY SPEED - KILN/COOLER"].mean())

In [168]:
#ELIMINAMOS LAS FILAS CON VALORES NULOS PUES SON MUY POCAS
balfdparos=balfdparos.dropna()

In [169]:
#VOLVEMOS A CHECAR
balfdparos.isnull().sum()

date                                       0
CALC. ROTOWFDR FEEDRATE -KILN2             0
CALC. ROTOWFDR LOADING MATERIAL -KILN2     0
CALCINER BLOWER CURRENT -KILN2             0
CALCINER FEEDER BLOWER PRESSURE -KILN2     0
FEED FLOWMETER ACTUAL FEEDRATE -KILN2      0
FEED ROTOWFDR FEEDRATE -KILN2              0
K2 TIM FEEDRATE                            0
KILN BURNER WFDR SPEED -KILN2              0
KILN FEEDER BLOWER CURRENT -KILN2          0
KILN FEEDER BLOWER PRESSURE -KILN2         0
KLN BURNER WFDR LOADING MATERIAL -KILN2    0
KLN GAS BMS FLOW -KILN2                    0
MAIN BURNR ROTARY SPEED - KILN/COOLER      0
MAIN COKE FEED RATE -KILN2                 0
PYROCLON DUCT TEMPERATURE -KILN2           0
SCHENCK WOOD TPH                           0
TERTIARY AIR TO CALCINER TEMP. -KILN2      0
ano                                        0
mes                                        0
dia                                        0
hora                                       0
minuto    

In [170]:
#GUARDAMOS NUESTRO ARCHIVO PREPARADO PARA APLICAR UN MODELO DE MACHINE LEARNING
balfdparos.to_csv("balFUELDOSERconparos.csv", index=False)